In [57]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
import re
from sklearn.pipeline import Pipeline
import nltk
from nltk.stem.snowball import SnowballStemmer
from sklearn.metrics import f1_score
from nltk.corpus import stopwords
import pickle
import tweepy
from IPython.display import clear_output
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV

In [58]:
rf_df = pd.read_csv('data/full/rf_preproc_train.csv')

In [3]:
rf_df = rf_df[['user_id', 'avg_fav', 'avg_hash', 'avg_len', 'avg_ret', 'default_profile',
       'fake_words_score', 'favourites_count', 'followers_count', 'freq',
       'friends_count', 'genuine_words_score', 'listed_count', 'max_fav',
       'max_hash', 'max_len', 'max_ret', 'media_perc', 'min_fav', 'min_hash',
       'min_len', 'min_ret', 'porn_words_score',
       'profile_use_background_image', 'prop_words_score', 'quote_perc',
       'ret_perc', 'spam_words_score', 'statuses_count',
       'tweet_intradistance', 'url', 'url_intradistance', 'url_perc',
       'description_len', 'name_len', 'screen_name_len', 'age','target']]

In [4]:
X_train, X_val, y_train, y_val = train_test_split(rf_df.drop(columns=['target']), rf_df['target'], test_size=0.2, random_state=42)

In [12]:
rf = RandomForestClassifier(n_estimators=30, criterion='gini', max_depth=24) #max_depth=24

In [13]:
cv = cross_validate(rf, X_train.drop(columns=['user_id']), y_train, cv=10, scoring='f1_macro')

In [14]:
cv['test_score'].mean()

0.9582357698077709

In [15]:
rf.fit(X_train.drop(columns=['user_id']), y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=24, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=30, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [16]:
rf_pred = rf.predict_proba(X_val.drop(columns=['user_id']))

In [17]:
rf.predict_proba(X_val.drop(columns=['user_id']).iloc[0:1])

array([[0.93333333, 0.        , 0.        , 0.03333333, 0.03333333]])

In [18]:
X_val.iloc[0:1]['user_id']

4347    610392907
Name: user_id, dtype: int64

In [19]:
rf_prob = pd.DataFrame(rf_pred, columns=['rf_0', 'rf_1', 'rf_2', 'rf_3', 'rf_4'])

In [20]:
X_val.reset_index(inplace=True, drop=True)

In [21]:
rf_prob.reset_index(inplace=True, drop=True)

In [22]:
y_val.reset_index(inplace=True, drop=True)

In [23]:
rf_prob['user_id'] = X_val['user_id']

In [24]:
rf_prob['target'] = y_val

## Random Forest df è pronto

In [25]:
users = pd.read_csv('data/full/train.csv')

In [26]:
users = users[['user_id','target']]

In [27]:
tweets = pd.concat([pd.read_csv('data/propaganda/tweets.csv', sep='\t')[['user_id','full_text']], \
          pd.read_csv('data/porn/tweets.csv', sep='\t')[['user_id','full_text']], \
          pd.read_csv('data/spam/tweets.csv', sep='\t')[['user_id','full_text']], \
          pd.read_csv('data/fake_followers/tweets.csv', sep='\t')[['user_id','full_text']], \
          pd.read_csv('data/genuine/tweets.csv', sep='\t')[['user_id','full_text']]])

/home/matteo/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/matteo/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,8,18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/matteo/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,4,8,11,18,19,20,23,26) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/matteo/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,8,31,32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  int

In [28]:
tweets = tweets.drop_duplicates()

In [29]:
tweets = pd.merge(tweets, users, on='user_id')

In [30]:
mask = np.in1d(tweets.user_id, X_train.user_id)

In [31]:
val_mask = np.in1d(tweets.user_id, X_val.user_id)

In [32]:
nb_train = tweets[mask]

In [33]:
nb_val = tweets[val_mask]

In [34]:
def remove_rt(x):
    if 'RT @' in x:
        try:
            return x[x.find(':')+2:]
        except:
            return x
    else:
        return x

In [35]:
stop_words = stopwords.words('english')

def remove_stop(x):
    return [word for word in x.split() if word not in stop_words]

In [36]:
nb_train['full_text'] = nb_train['full_text'].apply(lambda x: remove_rt(x))
nb_train['full_text'] = nb_train['full_text'].apply(lambda x: re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', x))
nb_train['full_text'] = nb_train['full_text'].apply(lambda x: re.sub(r'[^\w\s]','',x))
nb_train['full_text'] = nb_train['full_text'].apply(lambda x: x.lower())
nb_train['full_text'] = nb_train['full_text'].apply(lambda x: remove_stop(x))
nb_train['full_text'] = nb_train['full_text'].astype(str)
nb_train = nb_train[nb_train['full_text']!='[]']

/home/matteo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/matteo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/matteo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

In [37]:
nb_val['full_text'] = nb_val['full_text'].apply(lambda x: remove_rt(x))
nb_val['full_text'] = nb_val['full_text'].apply(lambda x: re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', x))
nb_val['full_text'] = nb_val['full_text'].apply(lambda x: re.sub(r'[^\w\s]','',x))
nb_val['full_text'] = nb_val['full_text'].apply(lambda x: x.lower())
nb_val['full_text'] = nb_val['full_text'].apply(lambda x: remove_stop(x))
nb_val['full_text'] = nb_val['full_text'].astype(str)
nb_val = nb_val[nb_val['full_text']!='[]']

/home/matteo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/matteo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/matteo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

In [38]:
stemmer = SnowballStemmer("english", ignore_stopwords=True)

class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(CountVectorizer, self).build_analyzer()
        return lambda doc:(stemmer.stem(w) for w in analyzer(doc))


stem_vectorizer = StemmedCountVectorizer(stemmer)

pipeline = Pipeline([('vect', stem_vectorizer), 
                     ('tfidf', TfidfTransformer()), 
                     ('mnb', MultinomialNB(fit_prior=False))])

clf = pipeline.fit(nb_train['full_text'], nb_train['target'])

In [39]:
predictions = []
i = 0

for usr in X_val['user_id']:
    
    i+=1
    clear_output()
    print(i)
    tweets_list = nb_val[nb_val.user_id == usr]['full_text']
    tweets_list = tweets_list[tweets_list.notnull()]
    if len(tweets_list[tweets_list.notnull()]) != 0:
        pred = clf.predict_proba(tweets_list)
        predictions.append(np.mean(pred, axis=0))
    else:
        predictions.append(np.array([0.2,0.2,0.2,0.2,0.2]))

5022


In [40]:
prob = pd.DataFrame(predictions, columns=['nb_0', 'nb_1', 'nb_2', 'nb_3', 'nb_4'])

In [41]:
prob['target'] = y_val

In [42]:
prob['user_id'] = X_val['user_id']

In [43]:
# prob.to_csv('data/full/predictions1.csv')

In [44]:
stack_df = rf_prob.merge(prob, on=['user_id' , 'target'])

In [45]:
stack_df = stack_df.reindex_axis(sorted(stack_df.columns), axis=1)

/home/matteo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  """Entry point for launching an IPython kernel.


In [46]:
stack_df.head()

,nb_0,nb_1,nb_2,nb_3,nb_4,rf_0,rf_1,rf_2,rf_3,rf_4,target,user_id
0,0.200000,0.200000,0.200000,0.200000,0.200000,0.933333,0.0,0.0,0.033333,0.033333,0,610392907
1,0.200000,0.200000,0.200000,0.200000,0.200000,0.966667,0.0,0.0,0.033333,0.000000,0,239615243
2,0.022305,0.844103,0.065607,0.003753,0.064231,0.000000,1.0,0.0,0.000000,0.000000,1,705441738391838720
3,0.013702,0.883903,0.049563,0.000985,0.051848,0.000000,0.9,0.0,0.000000,0.100000,1,32871086
4,0.200000,0.200000,0.200000,0.200000,0.200000,0.966667,0.0,0.0,0.000000,0.033333,0,185036273


In [47]:
# stack_df.to_csv('data/full/logreg_train.csv')

## Metamodel training

In [48]:
lr = LogisticRegressionCV(penalty='l2', solver='saga', max_iter=1000)

In [49]:
cv = cross_validate(lr, stack_df.drop(columns=['user_id', 'target']), stack_df['target'], cv=10, scoring='f1_macro')

In [50]:
cv['test_score'].mean()

0.9624819658554985

In [51]:
scores = lr.fit(stack_df.drop(columns=['user_id', 'target']),  stack_df['target']).scores_

In [52]:
scores

{0: array([[0.71641791, 0.71641791, 0.96835821, 0.97791045, 0.97850746,
         0.97970149, 0.98149254, 0.98089552, 0.98089552, 0.98089552],
        [0.71641791, 0.71641791, 0.96537313, 0.97970149, 0.97910448,
         0.98029851, 0.98029851, 0.98029851, 0.98029851, 0.98029851],
        [0.71650718, 0.71650718, 0.96889952, 0.97906699, 0.98145933,
         0.9826555 , 0.98325359, 0.98325359, 0.98325359, 0.98325359]]),
 1: array([[0.88656716, 0.88656716, 0.88656716, 0.9838806 , 0.98686567,
         0.98746269, 0.98746269, 0.98746269, 0.98746269, 0.98746269],
        [0.88656716, 0.88656716, 0.88656716, 0.98746269, 0.99223881,
         0.99402985, 0.99343284, 0.99343284, 0.99343284, 0.99343284],
        [0.88636364, 0.88636364, 0.88636364, 0.98803828, 0.99222488,
         0.99222488, 0.99043062, 0.99043062, 0.99043062, 0.99043062]]),
 2: array([[0.78089552, 0.78089552, 0.96955224, 0.98149254, 0.98507463,
         0.98567164, 0.98507463, 0.98507463, 0.98507463, 0.98507463],
        [0.780

In [54]:
lr.predict_proba(stack_df.drop(columns=['user_id', 'target']).iloc[1:2])

array([[0.98146432, 0.00241668, 0.00450503, 0.00624864, 0.00536533]])

In [56]:
pickle.dump( lr, open( "../scripts/lr.model", "wb" ) )